# ML Week 5 - Decision Trees
##### Today we will build our second Machine Learning application from scratch (but we have already built one, so we are pros now)
##### We will be building a supervised algorithm called a "Decision Tree". It can learn to efficiently classify objects using pre-determined labels.

#### Much of this notebook is inspired by these two blog posts:
* https://victorzhou.com/blog/intro-to-random-forests/
* https://victorzhou.com/blog/gini-impurity/

### Datset
We will again re-use our simple tools dataset from week 3.

Next week we will apply a decision tree to a real world dataset

### How it works:

<div>
<img src="decisionTree.png" width="500"/>
</div>

## Importing Common Packages
##### You know the routine...

In [2]:
import numpy as np
import pandas as pd

## Read in our dataset

In [3]:
# download tool data and read it into a dataframe
df = pd.read_csv("./tools.csv")

In [4]:
df

,Length,Width,Thickness,SharpEdgeWidth,Culture
0,5.1,3.5,1.4,0.2,Bear Culture
1,4.9,3.0,1.4,0.2,Bear Culture
2,4.7,3.2,1.3,0.2,Bear Culture
3,4.6,3.1,1.5,0.2,Bear Culture
4,5.0,3.6,1.4,0.2,Bear Culture
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Beaver Culture
146,6.3,2.5,5.0,1.9,Beaver Culture
147,6.5,3.0,5.2,2.0,Beaver Culture
148,6.2,3.4,5.4,2.3,Beaver Culture


## Split it Into Data (X) and Labels (Y)
##### This time we need to have the labels also as int values (so we use the label encoder function)

In [5]:
from sklearn.preprocessing import LabelEncoder

X = df[["Length", "Width", "Thickness", "SharpEdgeWidth"]].to_numpy()
Y = df['Culture'].to_numpy()

# Cleaner way:
# X = df.iloc[:,:-1].to_numpy()
# Y = df.iloc[:,-1].to_numpy()

Y_strings = Y

le = LabelEncoder()
Y = le.fit_transform(Y)

Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

##### We may need to recall which number correponsds to which label, so we save them in a dict

In [6]:
label_dict = {}
for i in range(len(Y)):
    label_dict[int(Y[i])] = Y_strings[i]

In [7]:
label_dict

{0: 'Bear Culture', 2: 'Owl Culture', 1: 'Beaver Culture'}

##### Our data is now ready

In [8]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

## We want a measure of how "impure" our dataset is
##### For this we will use "Gini Impurity". A perfect dataset with only 1 class would have impurity = 0, while a huge dataset where every datapoint is a different class would have be very impure (gini impurity = almost 1).

##### The way to calculate Gini Impurity is asking "if I randomly choose a data point, and then randomly classify it, what is the chance I've classified it wrong?"

##### Note: right now we only need the labels (Y)

In [9]:
def gini(Y:np.array) -> float:
    gini = 0
    for y in set(Y):  # For each class...
        probability = len(Y[Y==y])/len(Y)  # What is the probability of choosing that class?
        gini += probability*(1-probability)  # What's the probability we classify it incorrectly?
    return gini

In [10]:
# Let's see the Gini Impurity of our current dataset

gini(Y)

0.6666666666666667

## Deciding the best way to split a dataset

##### We will now make a function that, given a dataset (data X and corresponding labels Y) decides the best way to split the data

##### It will do this by testing all possible ways of splitting the dataset, and seeing which way reduces the impurity the most

##### Note: There are much more clever and efficient ways of doing this, but the "brute force" method is the easiest to visualize

In [11]:
# The function will return an integer and float.
# This is interpreted as: split the data X based on whether column "int" is smaller or larger than "float"
# ie. If the function returns 2, 3.5 this means all rows where column 2 is smaller than 3.5 should form one sub_dataset and all rows where column 2 is greater or equal than 3.5 should go in another

def find_best_split(X:np.array, Y:np.array) -> tuple[int, float]:
    # We will try to find the best column and value for the split. For now we initialize them to be None:
    best_col = None
    best_split = None
    # The current impurity to "beat" is that of the entire dataset Y
    best_gini = gini(Y)

    # Now we check each column:
    for col in range(X.shape[1]):
        # Sort the column so we can check each split in order from smallest to largest
        # Note we don't check the first value (hence the [1:]) because there is no datapoint with a value less than this and it will cause errors
        X_sorted = sorted(set(X[:,col]))[1:]
        # Now we try splitting for each value in this column
        for x in X_sorted:
            # here we split the dataset:
            left_split = Y[X[:,col] < x]
            right_split = Y[X[:,col] >= x]
            # Then we calculate the new impurity
            # Note: It's easy to have a pure dataset if we have very few datapoints
            # Because of this we need to weight the left and right sides. If we have very few datapoints, this should be weighted less than if we have lots
            new_gini = (len(left_split)/len(Y))*gini(left_split) + (len(right_split)/len(Y))*gini(right_split)
            # If we have beat the previous gini, let's update everything:
            if new_gini < best_gini:
                best_gini = new_gini
                best_col = col
                best_split = x
                # print(best_col, best_split, best_gini)
    return best_col, best_split

            

In [12]:
# For our current dataset, we find that the impurity is most reduced if we split on column 2 at the value of 3.0

find_best_split(X,Y)

(2, np.float64(3.0))

## We now have everything we need to build our decision tree!

##### The plan:

##### See if our data has more than one class.
##### If it doesn't, split it using our "find_best_split" function (to minimize impurity), then recursively apply the decision_tree function to both the left and right splits.
##### If it does, we're done.

In [13]:
def decision_tree(X:np.array, Y:np.array, level=0) -> None:
    # If the data has more than 1 class:
    if len(set(Y)) > 1:
        # Find the best split
        best_col, best_split = find_best_split(X,Y)
        print(level, best_col, best_split)

        # Split the dataset using the best split
        left_X = X[X[:,best_col] < best_split]
        left_Y = Y[X[:,best_col] < best_split]
        right_X = X[X[:,best_col] >= best_split]
        right_Y = Y[X[:,best_col] >= best_split]

        # Now recursively apply decision trees to both sides:
        print("going left")
        decision_tree(left_X, left_Y, level=level+1)
        print("going right")
        decision_tree(right_X, right_Y, level=level+1)
        return None
    # If there is only a single class, we're done!
    else:
        print("leaf node reached with value " + str(Y[0]))
        return None

In [14]:
# Let's try it out on our data!
decision_tree(X,Y)

0 2 3.0
going left
leaf node reached with value 0
going right
1 3 1.8
going left
2 2 5.0
going left
3 3 1.7
going left
leaf node reached with value 2
going right
leaf node reached with value 1
going right
3 3 1.6
going left
leaf node reached with value 1
going right
4 0 7.2
going left
leaf node reached with value 2
going right
leaf node reached with value 1
going right
2 2 4.9
going left
3 0 6.0
going left
leaf node reached with value 2
going right
leaf node reached with value 1
going right
leaf node reached with value 1


## What if we want to apply the tree to a new tool?
##### In general a Decision Tree should be built using something called "object oriented coding"
##### It is possible to build it using if/else loops, although it's a bit awkward

In [15]:
def tree(x):
    if x[2] < 3.0:  # Level 0
        return 0
    else:
        if x[3] < 1.8:  # Level 1
            if x[2] < 5.0:  # Level 2
                if x[3] < 1.7:  # Level 3
                    return 2
                else:
                    return 1
            else:
                if x[3] < 1.6:  # Level 3
                    return 1
                else:
                    if x[0] < 7.2:  # Level 4
                        return 2
                    else:
                        return 1
        else:
            if x[2] < 4.9:  # Level 2
                if x[0] <  6.0:  # Level 3
                    return 2
                else:
                    return 1
            else:
                return 1

In [16]:
tool_class = tree([2,3,4,2])
label_dict[tool_class]


'Owl Culture'

# Exercises

## Exercise 1
##### A Decision Tree will always find a way to perfectly classify the data it's given, even if it takes 1000 splits. This can result in what's called "overfitting", where a decision tree learns the data "too well" and cannot generalize to other data.

##### To fix this, we generally split the dataset into two new datasets: a training set and a testing set.

##### Then, we create the decision tree with ONLY the train data, and see how well it performs on the test data.

##### This gives us an idea of how well it is generalizing, and we can play with the parametres to see if we can improve it's generalizability

##### In this exercise we will explore how to do this. You will likely need to consult google for many of these steps.

##### Exercise 1.1 : Read in the data again like we did in the beginning, and then shuffle the rows of the dataframe randomly in order to assure the classes are evenly mixed. Save this into numpy arrays X and Y like before

In [48]:
df = df.sample(frac=1, random_state=0)
X = df[["Length", "Width", "Thickness", "SharpEdgeWidth"]].to_numpy()
Y = df['Culture'].to_numpy()
Y = le.fit_transform(Y)

##### Exercise 1.2 : Now take the first 70% of your dataset and save it into X_train and Y_train. The remaining 30% you should save into X_test and Y_test

##### The 70/30 split is standard in ML, although case-dependant other ratios may be more appropriate.

In [49]:
split_index = int(len(X)*0.7)

X_train = X[:split_index,:]
Y_train = Y[:split_index]

X_test = X[split_index:,:]
Y_test = Y[split_index:]

##### Exercise 1.3 : Now rebuild your decision tree on only X_train and Y_train. You can do this by copying the if/else tree function above and changing the values and structure as needed.

In [50]:
decision_tree(X_train,Y_train)

0 2 3.3
going left
leaf node reached with value 0
going right
1 3 1.6
going left
leaf node reached with value 2
going right
2 3 1.8
going left
3 0 6.0
going left
leaf node reached with value 1
going right
4 0 7.2
going left
leaf node reached with value 2
going right
leaf node reached with value 1
going right
3 2 4.9
going left
4 0 6.0
going left
leaf node reached with value 2
going right
leaf node reached with value 1
going right
leaf node reached with value 1


In [51]:
def tree(x):
    if x[2] < 3.3:  # Level 0
        return 0
    else:
        if x[3] < 1.6:  # Level 1
            return 2
        else:
            if x[3] < 1.8:
                if x[0] < 6.0:
                    return 1
                else:
                    if x[0] < 7.2:
                        return 2
                    else:
                        return 1
            else:
                if x[2] < 4.9:
                    if x[0] < 6.0:
                        return 2
                    else:
                        return 1
                else:
                    return 1



##### Exercise 1.5 : Finally, compare the accuracy of the decision tree on X_test and Y_test. How did we do?

In [56]:
true = 0
for x,y in zip(X_test, Y_test):
    if tree(x) == y:
        true += 1

accuracy = true/len(Y_test)
print(accuracy*100)

91.11111111111111


## Exercise 2
##### To prevent the tree from overfitting we may want to stop it from going too "deep" (ie, only recursing a certain number of times).

##### How could we modify our decision_tree function to avoid this?

## Exercise 3
##### As you've probably imagined, decision trees, test/train splitting, and accuracy testing are all included in the kmeans library.
##### We will see these next week, but if you've completed the previous exercises, try already exploring the kmeans versions of decision trees and applying them to our data